<a href="https://colab.research.google.com/github/hsnam95/class2023Spring/blob/main/whisper_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Whisper demo** 
Whisper is a general-purpose speech recognition model. It is trained on a large dataset of diverse audio and is also a multi-task model that can perform multilingual speech recognition as well as speech translation and language identification. This demo cuts audio after around 30 secs.



### 1. Install dependencies

In [1]:
!git clone https://huggingface.co/spaces/openai/whisper
%cd whisper
!pip install -r requirements.txt
!pip install gradio

fatal: destination path 'whisper' already exists and is not an empty directory.
/content/whisper
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### 2. Run

Run this cell to show the gradio app

In [2]:
import os
os.system("pip install git+https://github.com/openai/whisper.git")
import gradio as gr
import whisper

from share_btn import community_icon_html, loading_icon_html, share_js
model = whisper.load_model("large-v2")

100%|█████████████████████████████████████| 2.87G/2.87G [00:55<00:00, 55.5MiB/s]


In [ ]:
def inference(audio):
    audio = whisper.load_audio(audio)
    audio = whisper.pad_or_trim(audio)
    
    mel = whisper.log_mel_spectrogram(audio).to(model.device)
    
    _, probs = model.detect_language(mel)
    
    options = whisper.DecodingOptions(fp16 = False)
    result = whisper.decode(model, mel, options)
    return result.text

block = gr.Blocks()

with block:
    audio = gr.Audio(
                source="microphone",
                type="filepath"
            )
    btn = gr.Button("Transcribe")
    text = gr.Textbox()
    btn.click(inference, inputs=audio, outputs=text)
block.launch()

In [1]:
!pip install transformers
import torch
from transformers import pipeline
# !pip install datasets
# from datasets import load_dataset


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.5 MB/s eta 0:00:00


In [3]:
# TPU로 설정하면 comment out the below
# device = "cuda:0" if torch.cuda.is_available() else "cpu" 
pipe = pipeline(
  "automatic-speech-recognition",
  model="openai/whisper-large-v2",
  chunk_length_s=30,
  # device=device,
  # generate_kwargs={"language":"zh"}
)

# ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
# sample = ds[0]["audio"]

# prediction = pipe(sample.copy(), batch_size=8)["chunks"]

# # we can also return timestamps for the predictions
# prediction = pipe(sample.copy(), batch_size=8, return_timestamps=True)["chunks"]


In [4]:
!pip install gradio
import gradio as gr

def inference(audio):
  prediction = pipe(audio, batch_size=8, return_timestamps=True)["chunks"]

  return prediction
  
block = gr.Blocks()
with block:
    audio = gr.Audio(
                source="microphone",
                type="filepath"
            )
    btn = gr.Button("Transcribe")
    text = gr.Textbox()
    btn.click(inference, inputs=audio, outputs=text)
block.launch()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 77.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.9/287.9 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.2/137.2 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 k

<IPython.core.display.Javascript object>